<!--Copyright © ZOMI 适用于[License](https://github.com/Infrasys-AI/AIInfra)版权许可-->

# CODE 03:多模态输出采样与控制

> 多模态生成模型正在改变我们创造和理解内容的方式，而采样策略则是控制这一过程的隐形艺术家。

在当前的人工智能浪潮中，多模态生成模型已经成为内容创作的重要工具。无论是通过文字描述生成图像，还是根据静态图像创建视频，这些模型都展现出了惊人的能力。

但你是否曾经好奇，为什么同样的输入提示，有时会产生令人惊艳的结果，有时却平平无奇？这背后往往取决于采样策略的选择。

今天我们将深入探讨温度（Temperature）和 Top-P（核采样）两种采样策略如何影响多模态生成模型的输出结果，以及如何通过调整这些参数来平衡生成结果的**多样性**、**创造性**和**质量**。

### 1. Temperature 采样

温度参数或许是控制生成随机性最直观的方式。它在数学上调整了模型输出概率分布的平滑程度。

给定原始概率分布 $P(x_i|x_{<i})$，应用温度参数 $T$ 后的新概率分布为：

$$\hat{P}(x_i|x_{<i}) = \frac{\exp(\frac{z_i}{T})}{\sum_j \exp(\frac{z_j}{T})}$$

其中 $z_i$ 是模型输出的 logits 值。

**温度参数的影响**：

- 当 $T > 1$ 时，概率分布变得更加平滑，生成结果更加多样但可能不够准确
- 当 $T < 1$ 时，概率分布更加尖锐，生成结果更加确定但可能缺乏创造性

为了更好地理解这一概念，让我们通过代码来实现温度采样的效果：

In [1]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"
import torch
import torch.nn.functional as F

def apply_temperature(logits, temperature):
    if temperature > 0:
        # 应用温度缩放
        scaled_logits = logits / temperature
        # 重新计算 softmax 概率
        probs = F.softmax(scaled_logits, dim=-1)
        return probs
    else:
        raise ValueError("温度参数必须大于 0")

# 示例用法
logits = torch.tensor([2.0, 1.0, 0.5, 0.2])
print("原始概率:", F.softmax(logits, dim=-1).numpy())

for temp in [0.5, 1.0, 2.0]:
    temp_probs = apply_temperature(logits, temp)
    print(f"温度 {temp} 后的概率: {temp_probs.numpy()}")

原始概率: [0.56937605 0.20946175 0.12704498 0.09411723]
温度 0.5 后的概率: [0.8247789  0.11162169 0.04106332 0.02253603]
温度 1.0 后的概率: [0.56937605 0.20946175 0.12704498 0.09411723]
温度 2.0 后的概率: [0.4023389  0.24403088 0.19005144 0.1635788 ]


这段代码展示了温度参数如何影响概率分布。当温度值较低时（如 0.5），概率分布更加集中，模型更倾向于选择最高概率的选项；而当温度值较高时（如 2.0），概率分布更加平滑，模型的选择更加多样化。

## 2. Top-P 采样

Top-P 采样，也称为核采样，选择概率累积超过阈值 p 的最小可能词元集合，然后从这个集合中重新归一化概率并采样。

形式上，给定概率分布 $P$ 和阈值 $p ∈ (0, 1]$，我们按概率降序排列，找到最小的集合 $S$ 使得：

$$\sum_{x_i \in S} P(x_i) \geq p$$

然后从集合 $S$ 中按照重新归一化的概率进行采样。

In [2]:
def top_p_sampling(probs, p):
    # 对概率进行排序
    sorted_probs, indices = torch.sort(probs, descending=True)
    # 计算累积概率
    cumulative_probs = torch.cumsum(sorted_probs, dim=-1)
    
    # 移除累积概率超过 p 的部分
    indices_to_remove = cumulative_probs > p
    # 确保至少保留一个 token
    indices_to_remove[..., 1:] = indices_to_remove[..., :-1].clone()
    indices_to_remove[..., 0] = 0
    
    # 将需要移除的 token 概率设为 0
    sorted_probs[indices_to_remove] = 0
    # 重新归一化概率
    sorted_probs /= sorted_probs.sum()
    
    # 恢复到原始顺序
    return sorted_probs.scatter(-1, indices, sorted_probs)

# 示例用法
probs = torch.tensor([0.4, 0.3, 0.2, 0.1])
print("原始概率:", probs.numpy())

for p in [0.7, 0.9]:
    top_p_probs = top_p_sampling(probs, p)
    print(f"Top-P (p={p}) 后的概率: {top_p_probs.numpy()}")

原始概率: [0.4 0.3 0.2 0.1]
Top-P (p=0.7) 后的概率: [0.57142854 0.4285714  0.         0.        ]
Top-P (p=0.9) 后的概率: [0.44444445 0.33333334 0.22222222 0.        ]


Top-P 采样的优势在于它能够动态调整候选集的大小，既保证了多样性，又避免了选择概率极低的选项。

## 3. 文生图应用实践

了解了采样策略的基本原理后，让我们看看它们在实际的多模态生成任务中如何应用。我们将使用 Stable Diffusion 模型进行文本到图像的生成实验。

首先，我们需要设置环境并加载模型：

In [4]:
from diffusers import StableDiffusionXLPipeline
import torch

# 加载 SDXL 管道
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", 
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16"
)

# 将管道移动到 GPU（如果可用）
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = pipe.to(device)

/root/.pyenv/versions/3.11.1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00,  9.33it/s]


虽然 Stable Diffusion 本身不直接暴露温度参数，但我们可以通过修改生成过程中的随机性来模拟类似效果。实际上，在扩散模型中，类似的随机性控制可以通过调整 guidance_scale 和随机种子来实现。

In [6]:
def generate_images_with_variation(prompt, num_images=4, guidance_scale=7.5):
    images = []
    for i in range(num_images):
        # 使用不同的随机种子
        generator = torch.Generator(device=device).manual_seed(i)
        
        # 生成图像
        image = pipe(
            prompt=prompt,
            num_inference_steps=30,
            guidance_scale=guidance_scale,
            generator=generator,
        ).images[0]
        
        images.append(image)
    
    return images

# 设置生成参数
prompt = "一个美丽的日落海滩，有椰子树和金色的沙滩"
negative_prompt = "模糊，失真，低质量"

# 生成图像
images = generate_images_with_variation(prompt)
# 保存图像
for i, img in enumerate(images):
    img.save(f"./workspace/images/CODE03generated_{i}.png")


  3%|▎         | 1/30 [00:00<00:12,  2.37it/s]

100%|██████████| 30/30 [00:22<00:00,  1.34it/s]
/root/.pyenv/versions/3.11.1/lib/python3.11/site-packages/diffusers/pipelines/stable_diffusion_xl/pipeline_stable_diffusion_xl.py:748: FutureWarning: `upcast_vae` is deprecated and will be removed in version 1.0.0. `upcast_vae` is deprecated. Please use `pipe.vae.to(torch.float32)`. For more details, please refer to: https://github.com/huggingface/diffusers/pull/12619#issue-3606633695.
  deprecate(
100%|██████████| 30/30 [00:23<00:00,  1.29it/s]


在实际应用中，我们往往需要更精细的控制，而不仅仅是调整随机种子。多模态生成的可控性技术正在不断发展，例如通过 ControlNet 实现空间精准定位，通过 LoRA 注入特定规则，以及通过 CLIP 进行情感校准等方法。

## 4. 多模态实验

除了文生图应用，采样策略也对多模态语言模型的输出有重要影响。让我们以视觉语言模型为例，看看不同采样参数如何影响模型生成的描述。

我们将使用 Qwen-VL 模型进行图文对话实验，观察不同温度和 Top-P 参数如何影响生成的图像描述。

In [3]:
import os
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq
# ======================
# 模型路径
# ======================
MODEL_PATH = "Qwen/Qwen2.5-VL-7B-Instruct"

# ======================
# 自动下载 processor 和模型
# ======================
print(">>> 加载 Processor ...")
processor = AutoProcessor.from_pretrained(MODEL_PATH, trust_remote_code=True)

print(">>> 加载 Model（FP16 + CUDA） ...")
model = AutoModelForVision2Seq.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.float16,
    device_map="cuda:0",   # 自动放到GPU
    trust_remote_code=True,
)

# ======================
# 加载图片
# ======================
def load_image(path):
    return Image.open(path).convert("RGB")


# ======================
# 生成描述（多组 sampling）
# ======================
def generate_descriptions(image_path, prompt_text, sampling_configs):
    image = load_image(image_path)

    results = []
    for cfg in sampling_configs:
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "image", "image": image},
                    {"type": "text", "text": prompt_text},
                ],
            }
        ]

        # 步骤1：生成 chat 模板文本
        chat_text = processor.apply_chat_template(
            messages,
            tokenize=False,        # or tokenize=True if你想直接tokenize
            add_generation_prompt=True,
            return_dict=False,     # 默认返回字符串
        )

        # 步骤2：把文本和图片一起传入 processor
        inputs = processor(
            text=[chat_text],           # 注意：这里是 list of str, 因为这个版本的Qwen需要这种数据
            images=[image],             # list of PIL Images 对应
            padding=True,
            return_tensors="pt"
        )
        # 移动到设备 + dtype
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        # 生成
        output = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=True,
            temperature=cfg["temperature"],
            top_p=cfg["top_p"],
        )

        # decode
        # 如果你用的是 batch，那么用 batch_decode
        text = processor.batch_decode(output, skip_special_tokens=True)[0]
        results.append((cfg["name"], text))

    return results



# ======================
# 定义采样配置（可改）
# ======================
sampling_configs = [
    {"temperature": 0.1, "top_p": 0.9, "name": "低温度 / 高Top-P"},
    {"temperature": 0.7, "top_p": 0.9, "name": "中温度 / 高Top-P"},
    {"temperature": 1.2, "top_p": 0.9, "name": "高温度 / 高Top-P"},
    {"temperature": 0.7, "top_p": 0.3, "name": "中温度 / 低Top-P"},
]


# ======================
# 运行（你只要改图片路径）
# ======================
image_path = "./workspace/images/CODE03generated_0.png"
prompt_text = "描述这张图片的内容、场景和细节"

descriptions = generate_descriptions(image_path, prompt_text, sampling_configs)

# ======================
# 输出结果
# ======================
print("\n====== 最终结果汇总 ======")
for name, text in descriptions:
    print(f"\n>>> {name}：\n{text}\n")


/root/.pyenv/versions/3.11.1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


>>> 加载 Processor ...


Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 12018.06it/s]
The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.
Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 10034.22it/s]


>>> 加载 Model（FP16 + CUDA） ...


/root/.pyenv/versions/3.11.1/lib/python3.11/site-packages/transformers/models/auto/modeling_auto.py:2242: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 5/5 [01:09<00:00, 13.85s/it]



====== 最终结果汇总 ======

>>> 低温度 / 高Top-P：
system
You are a helpful assistant.
user
描述这张图片的内容、场景和细节
assistant
这张图片描绘了一位身穿传统服饰的女子，背景是一个充满诗意的自然景观。女子的服饰华丽，颜色以白色为主，带有粉色和橙色的装饰，显得非常优雅。她的长发披肩，姿态优雅，似乎在欣赏周围的景色。

背景中有一棵大树，树上开满了粉红色的花朵，可能是樱花或桃花，给人一种春天的感觉。树干粗壮，枝叶繁茂，与女子的服饰形成了鲜明的对比。远处可以看到一些山峦，山间云雾缭绕，增添了几分神秘感。地面上有几片落叶，可能是秋天的景象，但整体色调仍然偏向温暖的色彩。

整个画面充满了古典美，仿佛将观众带入了一个古代的山水画中，展现了人与自然和谐共处的美好画面。


>>> 中温度 / 高Top-P：
system
You are a helpful assistant.
user
描述这张图片的内容、场景和细节
assistant
这张图片描绘了一位身穿传统汉服的女性，站在一个宁静的湖边。她的长发披肩，衣袂飘逸，背景中有一棵开满粉色花朵的大树，树枝错综复杂，与湖水相映成趣。

1. **人物**：图片中的女子身着一袭精致的白色汉服，裙摆宽大，显得非常优雅。她的姿态优雅，右手轻轻抚摸着树干，仿佛在与自然对话。

2. **环境**：画面背景是一个宁静的湖景，湖面波光粼粼，远处的山脉隐约可见，增添了几分神秘感。树木上盛开的粉色花朵为整个画面增添了生机与活力，与湖水的颜色形成鲜明对比。

3. **色彩**：整体色调柔和，以粉色、白色和灰色为主，给人一种宁静和谐的感觉。粉色的花朵、白色的衣裳和灰色的树木相互映衬，营造出一种古典而梦幻的氛围。

4. **细节**：树叶随风飘落，仿佛在诉说着季节的变换，增加了画面的层次感。此外，女子的手势也传达出一种宁静与淡然的心境，仿佛置身于世外桃源。

这张图片通过细腻的


>>> 高温度 / 高Top-P：
system
You are a helpful assistant.
user
描述这张图片的内容、场景和细节
assistant
这张图片是一幅古风插画，描绘了一位身穿传统汉服的女子站在自然景观中。女子的服饰精致华丽，颜色为浅灰色和橙色相间，长发

通过这个实验，我们可以观察到不同采样配置下模型生成的描述有何不同。低温度配置往往产生更加保守和确定的描述，而高温度配置则可能产生更加创造性和多样化的描述，但也可能增加不相关或虚构内容的风险。

## 5. 评估生成结果

评估多模态生成结果的质量是一个复杂的任务，需要从多个维度进行考量。常用的评估指标包括：

1.  **模态对齐度（MDA）**：衡量生成内容与输入提示之间的一致性。
2.  **细节保真度（DF）**：评估生成内容的细节丰富程度和准确性。
3.  **多样性**：衡量不同生成结果之间的差异程度。
4.  **创造性**：评估生成内容的新颖性和创新程度。

我们可以通过计算一些定量指标来评估生成结果的多样性：

In [5]:
from collections import Counter
import math

def calculate_diversity(descriptions):
    """
    计算生成描述的多样性指标
    """
    all_words = []
    for item in descriptions:
        # 假设每个 item 是 (标签, 描述字符串)
        if isinstance(item, tuple) and len(item) >= 2:
            desc = item[1]
        if not isinstance(desc, str):
            raise ValueError(f"Expected description to be str, but got {type(desc).__name__}: {desc!r}")
        words = desc.lower().split()
        all_words.extend(words)
    
    # 计算词汇总量和唯一词汇量
    total_words = len(all_words)
    unique_words = len(set(all_words))
    lexical_diversity = unique_words / total_words if total_words > 0 else 0
    print(lexical_diversity)
    return {
        "lexical_diversity": lexical_diversity,
        "unique_words": unique_words,
        "total_words": total_words
    }

# 计算生成描述的多样性
diversity_metrics = calculate_diversity(descriptions)

0.5178571428571429


除了这些定量指标，人工评估仍然是评估生成质量的重要方式，特别是对于创造性和审美价值的判断。

## 6. 总结与思考

采样策略在多模态生成中扮演着至关重要的角色，它们像是隐形的艺术家，默默地影响着生成结果的多样性、创造性和质量。通过理解和掌握温度参数和 Top-P 采样等策略，我们能够更好地驾驭多模态生成模型，创造出更加符合期望的内容。

需要注意的是，参数调整并非万能，它需要在模型能力、任务需求和使用场景之间找到平衡点。有时候，**创造力的提升可能会以降低精确性为代价**，而**过于追求确定性又可能抑制创新**。这正是多模态生成既是一门科学也是一门艺术的原因。

希望本文能够为你理解和应用采样策略提供有益的指导，帮助你在多模态生成的探索之旅中走得更远。记住，最好的参数配置往往来自于不断的实验和调整，而不是一成不变的公式。